In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
## BADD-- supressing f1 score ill-defined and being set to 0.0 in labels with no true samples warning
import warnings
warnings.filterwarnings("ignore")

In [4]:
from pathlib import Path
import pandas as pd

In [5]:
import  datatset
from torch.utils.data import DataLoader

In [6]:
# DataLoader

In [7]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [8]:
def over_sample_class(x):
    if len(x) < 20:
        return x.sample(n=20, replace=True) 
    return x

In [9]:
len(df_train)

1030747

In [10]:
new_df = df_train.groupby('category_id').apply(over_sample_class).reset_index(drop = True)

In [11]:
counts  = new_df["category_id"].value_counts()

labels_aug = counts.keys()[counts < 21] # Get labels that have less then 9 exmaples

augment = new_df["category_id"].isin(labels_aug) # Get list of items to apply aug

In [12]:
new_df["augment"] = augment

In [13]:
np.sum(new_df.augment)

448020

In [14]:
label_counts  = df["category_id"].value_counts()

In [15]:
# Update class labels
classes = new_df["category_id"].unique()
classes_map = {classes[i]:i for i in range(len(classes))}
new_labels = [classes_map[i] for i in new_df["category_id"]]
new_df["new_labels"] = new_labels

In [16]:
num_calsses = len(classes)

In [17]:
num_calsses

32093

In [18]:
from tqdm import tqdm_notebook

# there is a better way to do this
def sample(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        if validation_set.get(row["new_labels"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["new_labels"]] = validation_set.get(row["new_labels"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

In [19]:
validation = sample(new_df, n=3)

In [20]:
new_df["is_valid"] = validation

In [21]:
train_df, valid_df = new_df[~new_df["is_valid"]] , new_df[new_df["is_valid"]]

In [22]:
len(train_df), len(valid_df)

(1223678, 96279)

In [23]:
ds = datatset.HerbDataSet(train_df, TRAIN_PATH,256,label="new_labels")

In [24]:
data_loader = DataLoader(ds, batch_size=32, shuffle=True)

In [25]:
# Model

In [26]:
import model
import torchvision.models as models

In [27]:
m = model.get_model(models.resnet32(pretrained=True), p1=1, p2=1 device="gpu")

In [28]:
import torch.nn as nn
import torch

In [29]:
assert m.fc[4].out_features == num_calsses

In [36]:
# Tracking

In [37]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-1")

In [38]:
import metrics
import train

In [39]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

In [40]:
# Train

In [41]:
criterion = nn.CrossEntropyLoss()

In [42]:
# layer_lrs = [
#     {'params': base.layer1.parameters(), 'lr': 1e-4},
#     {'params': base.layer2.parameters(), 'lr': 1e-4},
#     {'params': base.layer3.parameters(), 'lr': 1e-3},
#     {'params': base.layer4.parameters(), 'lr': 1e-3},
#     {'params': base.fc.parameters(), 'lr': 1e-2}
# ]

In [43]:
optimizer = torch.optim.Adam(m.fc.parameters(), lr=1e-3) # only training final layers

In [44]:
model_path=TRAIN_PATH/"models/"

In [45]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 5, m, model_path )

In [46]:
! export CUDA_LAUNCH_BLOCKING=1

In [47]:
trainer.run()

Epoch 0/5



Epoch 1/5
